In [0]:
#bronze and silver path
bronze_path = "/mnt/Prajwal/medlian_arch/Bronze/"
silver_path = "/mnt/Prajwal/medlian_arch/Silver"

In [0]:
#Reading file from bronze layer
df = spark.read.format("delta").load(bronze_path)
display(df)

In [0]:
#checking the schema
df.printSchema()

In [0]:
#reformating the schema for CustomerID and OrderID as integer and OrderDate as date, Quantity as float

from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

df = df.withColumn("CustomerID", col("CustomerID").cast(IntegerType())) \
       .withColumn("OrderDate", to_date(col("OrderDate"), "yyyy-MM-dd")) \
       .withColumn("Quantity", col("Quantity").cast(FloatType()))

df.printSchema()


In [0]:
display(df)

In [0]:
display(df.count())

In [0]:
#calculate price + tax and naming it as TotalAmount
df = df.withColumn("TotalAmount", col("Price") + col("Tax"))

# Calculate additional metrics (e.g., TotalOrderValue)
df = df.withColumn("TotalOrderValue", col("Quantity") * col("Price"))

#dropping null values
df = df.dropna()

display(df)

In [0]:
#writing to silver layer with partion by orderdate
df.write.format("delta").partitionBy("OrderDate").mode("overwrite").save(silver_path)